In [1]:
# streamlit run main.py
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
import transformers
from transformers import AutoTokenizer, AutoModel
from functools import lru_cache
import numpy as np
import streamlit as st
import sys
import subprocess
import json
import os
from tqdm import tqdm



from pdf_parser import predict_luminosity_from_url, predict_if_lighting

/Users/varshinibalaji/Documents/DSProjects/product_classifier/lclassifier_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/varshinibalaji/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/varshinibalaji/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
test_data = pd.read_pickle("data/test_data.pkl")
test_data.head()

,id,url,is_lighting,parsed_text,processed_text,parsed_lines,predict_lighting,light_phrase_index,light_phrase,updated_predict_lighting
0,TP0,https://cdn.currentlighting.com/site/specsheet...,0,Synchron AC Power System Suggested Specificati...,synchron ac power system suggested specificati...,[synchron ac power system suggested specificat...,True,7.0,output the system shall be capable of powering...,True
1,TP1,https://cdn.currentlighting.com/site/specsheet...,0,Synchron AC Power System Suggested Specificati...,synchron ac power system suggested specificati...,[synchron ac power system suggested specificat...,True,7.0,output the system shall be capable of powering...,True
2,TP2,https://cdn.currentlighting.com/site/specsheet...,0,Synchron AC Power System Suggested Specificati...,synchron ac power system suggested specificati...,[synchron ac power system suggested specificat...,True,7.0,output the system shall be capable of powering...,True
3,TP3,https://www.assets.signify.com/is/content/Sign...,0,LXF / LXT Lexington Lumen Tables\nStreetworks\...,lxf / lxt lexington lumen tables\nstreetworks\...,"[lxf / lxt lexington lumen tables, streetworks...",True,0.0,lxf / lxt lexington lumen tables,True
4,TP4,https://www.assets.signify.com/is/content/Sign...,0,LXF / LXT Lexington Lumen Tables\nStreetworks\...,lxf / lxt lexington lumen tables\nstreetworks\...,"[lxf / lxt lexington lumen tables, streetworks...",True,0.0,lxf / lxt lexington lumen tables,True


In [3]:
# is_lighting, product_desc, light_phrase, confidence_score = predict_luminosity_from_url(product_name)
tqdm.pandas()

test_data['preprocessed_text'] = test_data['parsed_lines']

# test_data['model_prediction_meta'] = test_data['processed_text'].progress_apply(lambda x: predict_luminosity_from_url(x, True))

test_data['model_prediction_meta'] = test_data['url'].progress_apply(lambda x: predict_luminosity_from_url(x))

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 80/80 [09:55<00:00,  7.44s/it]


In [24]:
test_data['url'].iloc[30]

'https://www.usailighting.com/site/view.php?id=42369'

In [4]:
test_data['model_prediction'] = test_data['model_prediction_meta'].apply(lambda x: x[0])

test_data['product_desc'] = test_data['model_prediction_meta'].apply(lambda x: x[1])

test_data['light_phrase'] = test_data['model_prediction_meta'].apply(lambda x: x[2])

test_data['confidence_score'] = test_data['model_prediction_meta'].apply(lambda x: x[3])

In [7]:
test_data.groupby(['is_lighting', 'model_prediction'], dropna = False).size()

is_lighting  model_prediction
0            False               50
             True                10
1            False                8
             True                12
dtype: int64

In [6]:
correct = test_data.loc[(((test_data['model_prediction'] == False)&(test_data['is_lighting'] == 0))|((test_data['model_prediction']== True)&(test_data['is_lighting'] == 1)))]

print(f"correct predictions: {len(correct)}")

print(f"total: {len(test_data)}")

print(f"accuracy: {len(correct)/len(test_data)}")

print(f"precision: {len(test_data.loc[((test_data['model_prediction'] == True)&(test_data['is_lighting'] == 1))])/len(test_data.loc[((test_data['model_prediction'] == True))])}")

print(f"recall: {len(test_data.loc[((test_data['model_prediction'] == True)&(test_data['is_lighting'] == 1))])/len(test_data.loc[((test_data['is_lighting'] == 1))])}")

correct predictions: 62
total: 80
accuracy: 0.775
precision: 0.5454545454545454
recall: 0.6


#### Metrics

- correct predictions: 62
- total: 80
- accuracy: 77.5%
- precision: 55%
- recall: 60%

In [12]:
test_data.to_pickle("temp/test_data_predictions.pkl")

In [15]:
test_data.loc[test_data['is_lighting'] == 1, 'url'].iloc[4]

'https://www.assets.signify.com/is/content/Signify/7f3dac4eae71462583e0ad41006cad73'

In [8]:
pd.set_option('display.max_colwidth', None)

### observing false positives

In [14]:
test_data.loc[((test_data['is_lighting'] == 0)&(test_data['model_prediction'] == True)), 'url']

3     https://www.assets.signify.com/is/content/Signify/04110edf0a674d49b97ead5f01854f50
4     https://www.assets.signify.com/is/content/Signify/04110edf0a674d49b97ead5f01854f50
5     https://www.assets.signify.com/is/content/Signify/04110edf0a674d49b97ead5f01854f50
40                                   https://www.usailighting.com/site/view.php?id=42450
41                                   https://www.usailighting.com/site/view.php?id=42450
57                                   https://www.usailighting.com/site/view.php?id=42927
58                                   https://www.usailighting.com/site/view.php?id=42927
59                                   https://www.usailighting.com/site/view.php?id=42928
70                                    https://www.usailighting.com/site/view.php?id=5902
71                                    https://www.usailighting.com/site/view.php?id=5902
Name: url, dtype: object

### observing false negatives

In [13]:
test_data.loc[((test_data['is_lighting'] == 1)&(test_data['model_prediction'] == False)), 'url']

7                                                  https://www.assets.signify.com/is/content/Signify/6f5854d9da7f40858c29ad41007d0a7f
8                                                  https://www.assets.signify.com/is/content/Signify/6f5854d9da7f40858c29ad41007d0a7f
9                                                  https://www.assets.signify.com/is/content/Signify/6f5854d9da7f40858c29ad41007d0a7f
15                                                 https://www.assets.signify.com/is/content/Signify/ac1049ecda8c4cf98458ad41006cbaf0
16    https://www.cooperlighting.com/api/assets/v1/file/CLS/content/1c22383ce4764a2da7deadbb010bd4e7/1c22383ce4764a2da7deadbb010bd4e7
17    https://www.cooperlighting.com/api/assets/v1/file/CLS/content/1c22383ce4764a2da7deadbb010bd4e7/1c22383ce4764a2da7deadbb010bd4e7
18    https://www.cooperlighting.com/api/assets/v1/file/CLS/content/1c22383ce4764a2da7deadbb010bd4e7/1c22383ce4764a2da7deadbb010bd4e7
26                                                            